## FFM Application with Python

### 1. Create Test Signals
To study how FFT works, I first created code to generate multiple sine signals using the following general form of a sine wave.
$$y = A \sin(2\pi f t + \phi)$$

In [1]:
import numpy as np
import os
import librosa
import IPython.display as ipd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
class sinWave:
    def __init__(self, **kwargs):
        self.A = kwargs.get('amp', 1)
        self.f = kwargs.get('freq', 1)
        self.p = kwargs.get('phase_shift', 0)
        self.endTime = kwargs.get('endTime', 1)
        self.sampleTime = kwargs.get('sampleTime', 0.01)

    def getDomain(self):
        return np.arange(0.0, self.endTime, self.sampleTime)
        
    def createSinWave(self,time):        
        return self.A * np.sin(2*np.pi*self.f*time + self.p)

The following code generate three different sine waves, each for the

$2sin(2\pi)$,

$5sin(3\cdot2\pi)$, and

$9sin(2\cdot2\pi)$.

In [3]:
sr = 1/22050 #standard 44.1 kHz
amplitude = 3
a4_freq = 440 
csharp5_freq = 554.365
e5_freq = 659.365

In [4]:
# generate sin waves
sin = sinWave(amp = amplitude, sampleTime=sr)
t = sin.getDomain()
sin.f = a4_freq
a4_y = sin.createSinWave(time=t)
sin.f = csharp5_freq
csharp5_y = sin.createSinWave(time=t)
sin.f = e5_freq
e5_y = sin.createSinWave(time=t)
a_chord = a4_y + csharp5_y + e5_y

#plot graph
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a4_y[:len(t)//20],
    mode='lines',
    name='A4',
    line=dict(color='rgba(173, 216, 230, 0.6)'), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=csharp5_y[:len(t)//20],
    mode='lines',
    name='C#5',
    line=dict(color='rgba(144, 238, 144, 0.6)'),  
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=e5_y[:len(t)//20],
    mode='lines',
    name='E5',
    line=dict(color='rgba(255, 182, 193, 0.6) '), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='A Chord',
    line=dict(color='rgba(221, 160, 221, 1.0)') 
))

fig.update_layout(
    title={
        'text':'A chord',
        'x': 0.5},
    xaxis_title='Time (s)',
    yaxis_title='Amplitude',
    font=dict(family="Arial", size=14)
)

fig.show()

In [5]:
a4_audio = ipd.Audio(data=a4_y, rate=1/sr)
csharp5_audio = ipd.Audio(data=csharp5_y, rate=1/sr)
e5_audio = ipd.Audio(data=e5_y, rate=1/sr)
ipd.Audio(data = a_chord, rate=1/sr)

In [6]:
noise_level = 0.5
noisy_a_chord = a_chord + noise_level * np.random.randn(len(a_chord))
sr = 1/22050 #standard 44.1 kHz
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=noisy_a_chord[:len(t)//20],
    mode='lines',
    name='noisy A chord',
    line=dict(color='LightBlue'), 
))
fig.add_trace(go.Scatter(
    
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='original A chord',
    line=dict(color='rgba(221, 160, 221, 1.0)'), 
))

fig.update_layout(
    title={
        'text':'Noisy A chord',
        'x': 0.5},
    xaxis_title='Time (s)',
    yaxis_title='Amplitude',
    font=dict(family="Arial", size=14)
)
fig.show()
ipd.Audio(data = noisy_a_chord, rate=1/sr)

In [7]:
N = len(a_chord)               # number of sampling points
sr = 1/sin.sampleTime    # sampling rate (# samples per second)
T = N/sr                            # total time
k = np.arange(N)                    # index for frequency [0-(n-1)]
freq = k/T                          # plotting points

In [8]:
a_chord_fft = np.fft.fft(a_chord) / N * 2
a_chord_fft_ps = (np.abs(a_chord_fft)**2) * (1/N**2)
a_chord_fft_ps_one_sided = a_chord_fft_ps[:N//2].copy()

noisy_a_chord_fft = np.fft.fft(noisy_a_chord) / N * 2
noisy_a_chord_fft_ps = (np.abs(noisy_a_chord_fft)**2) * (1/N**2)
noisy_a_chord_fft_ps_one_sided = noisy_a_chord_fft_ps[:N//2].copy()


In [9]:
graph_a_chord_fft = go.Figure()
graph_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(a_chord_fft[:1000])),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color='rgba(144, 238, 144, 0.5)'), 
))

graph_noisy_a_chord_fft = go.Figure()
graph_noisy_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(noisy_a_chord_fft[:1000])),
    mode='markers+lines',
    name='noisy A chord FFT',
    line=dict(color='rgba(221, 160, 221, 1.0)'), 
))

graph_noisy_a_chord_fft_zooo_in = go.Figure()
graph_noisy_a_chord_fft_zooo_in.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(noisy_a_chord_fft[:1000])),
    mode='markers+lines',
    name='noisy A chord FFT zoomed in',
    line=dict(color='rgba(221, 160, 221, 1.0)'), 
))

combined_graphs = make_subplots(rows=1, cols=3, subplot_titles=('A Chord FFT', 'Noisy A Chord FFT', 'noisy A chord FFT zoomed in'))
combined_graphs.add_traces(graph_a_chord_fft.data, rows=1, cols=1)
combined_graphs.add_traces(graph_noisy_a_chord_fft.data, rows=1, cols=2)
combined_graphs.add_traces(graph_noisy_a_chord_fft_zooo_in.data, rows=1, cols=3)
combined_graphs.update_layout(legend=dict(
    orientation='h',
    xanchor='center',
    x = 0.5,
    y = -0.1
),
title=dict(text='A Chord FFT in Magnitude Spectrum',
x = 0.5,
xanchor='center'
)
)

In [10]:
full_mask = abs(noisy_a_chord_fft) > 0.025
fhat = full_mask * noisy_a_chord_fft

graph_denoised_a_chord_fft = go.Figure()
graph_denoised_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(fhat[:1000])),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color='rgba(144, 238, 144, 0.5)'), 
))
graph_denoised_a_chord_fft.update_layout(
    title=dict(
        text='Denoised FFT',
        xanchor='center',
        x = 0.5,
        font_size=25,
    ),
    xaxis_title=dict(text='Frequency(Hz)', font_size=15),
    yaxis_title=dict(text='Amplitude', font_size=15),
)
graph_denoised_a_chord_fft.show()
denoised_achord_ifft = np.fft.ifft(fhat * N / 2)


In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=noisy_a_chord[:len(t)//20],
    mode='lines',
    name='noisy A chord',
    line=dict(color='rgba(221, 160, 221, 0.3)'), 
))
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=np.real(denoised_achord_ifft[:len(t)//20]),
    mode='lines',
    name='Denoised A chord',
    line=dict(color='rgba(255, 182, 193, 0.8)',
    width=4), 
))
fig.add_trace(go.Scatter(
    x=t[:len(t)//20],
    y=a_chord[:len(t)//20],
    mode='lines',
    name='original A chord',
    line=dict(color='rgba(144, 238, 144, 0.5)', 
    width=4), 
))

fig.update_layout(
    legend=dict(
    orientation='h',
    xanchor='center',
    x = 0.5,
    y = -0.1,
    font_size=20        
    ),
    title=dict(
        text='Filtered A chord sound (IFFT applied)',
        xanchor='center',
        x=0.5,
        font_size=20,
        font_weight='bold'
    )
)

In [12]:
ipd.Audio(data=np.real(denoised_achord_ifft), rate = sr)